# วันที่ 3: การสร้าง Dashboard และวิเคราะห์กรณีศึกษา
## ช่วงบ่าย (13.00 น.–16.00 น.): กรณีศึกษา – วิเคราะห์ข้อมูลจากสถานการณ์จริง

## 3. การวิเคราะห์ข้อมูลเบื้องต้น (ต่อ)

In [ ]:
# นำเข้าไลบรารีที่จำเป็น
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

# กำหนดให้แสดงกราฟในโน้ตบุ๊ก
%matplotlib inline
# กำหนดสไตล์ของกราฟ
plt.style.use('seaborn-v0_8')
# กำหนดให้แสดงคอลัมน์ทั้งหมดในดาต้าเฟรม
pd.set_option('display.max_columns', None)

# โหลดข้อมูลที่ได้สร้างไว้
# หากไม่มีไฟล์ CSV ให้รันโค้ดในไฟล์ 'วันที่3-บ่าย-2-การเตรียมข้อมูล.ipynb' ก่อน

try:
    products_df = pd.read_csv('power_bi_data/products.csv')
    customers_df = pd.read_csv('power_bi_data/customers.csv')
    stores_df = pd.read_csv('power_bi_data/stores.csv')
    sales_df = pd.read_csv('power_bi_data/sales.csv')
    
    # แปลงคอลัมน์ที่เกี่ยวกับวันที่ให้เป็น datetime
    sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])
    customers_df['registration_date'] = pd.to_datetime(customers_df['registration_date'])
    
    sales_df['month'] = sales_df['sale_date'].dt.strftime('%Y-%m')
    sales_df['quarter'] = sales_df['sale_date'].dt.to_period('Q').astype(str)
    sales_df['year'] = sales_df['sale_date'].dt.year
    sales_df['weekday'] = sales_df['sale_date'].dt.day_name()
    
    # เชื่อมข้อมูลการขายกับข้อมูลอื่นๆ
    sales_with_info = sales_df.merge(products_df, on='product_id', how='left')
    sales_with_info = sales_with_info.merge(customers_df, on='customer_id', how='left')
    sales_with_info = sales_with_info.merge(stores_df, on='store_id', how='left')
    
    print("โหลดข้อมูลเรียบร้อยแล้ว!")
except FileNotFoundError:
    print("ไม่พบไฟล์ข้อมูล โปรดรันโค้ดในไฟล์ 'วันที่3-บ่าย-2-การเตรียมข้อมูล.ipynb' ก่อน")

### 3.3 การวิเคราะห์พฤติกรรมลูกค้า

In [ ]:
# วิเคราะห์ยอดขายตามกลุ่มอายุและเพศ
sales_by_demographic = sales_with_info.groupby(['age_group', 'gender']).agg({
    'sales_amount': 'sum',
    'profit': 'sum',
    'customer_id': pd.Series.nunique,
    'transaction_id': pd.Series.nunique
}).reset_index()
sales_by_demographic.columns = ['age_group', 'gender', 'sales_amount', 'profit', 'num_customers', 'num_transactions']
sales_by_demographic['avg_spend_per_customer'] = sales_by_demographic['sales_amount'] / sales_by_demographic['num_customers']
sales_by_demographic = sales_by_demographic.sort_values(['age_group', 'gender'])

print("ยอดขายตามกลุ่มอายุและเพศ:")
display(sales_by_demographic)

# แสดงกราฟแท่งเปรียบเทียบยอดขายตามกลุ่มอายุและเพศ
plt.figure(figsize=(12, 6))
age_order = ['18-24', '25-34', '35-44', '45-54', '55+']
pivot_data = sales_by_demographic.pivot(index='age_group', columns='gender', values='sales_amount')
pivot_data = pivot_data.reindex(age_order)
pivot_data.plot(kind='bar', width=0.7)
plt.title('ยอดขายตามกลุ่มอายุและเพศ')
plt.xlabel('กลุ่มอายุ')
plt.ylabel('ยอดขาย (บาท)')
plt.legend(title='เพศ')
plt.tight_layout()
plt.show()

In [ ]:
# แสดงกราฟแท่งเปรียบเทียบค่าใช้จ่ายเฉลี่ยต่อลูกค้าตามกลุ่มอายุและเพศ
plt.figure(figsize=(12, 6))
avg_spend_pivot = sales_by_demographic.pivot(index='age_group', columns='gender', values='avg_spend_per_customer')
avg_spend_pivot = avg_spend_pivot.reindex(age_order)
avg_spend_pivot.plot(kind='bar', width=0.7)
plt.title('ค่าใช้จ่ายเฉลี่ยต่อลูกค้าตามกลุ่มอายุและเพศ')
plt.xlabel('กลุ่มอายุ')
plt.ylabel('ค่าใช้จ่ายเฉลี่ยต่อลูกค้า (บาท)')
plt.legend(title='เพศ')
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์รูปแบบการซื้อตามวันในสัปดาห์
sales_by_weekday = sales_df.groupby('weekday').agg({
    'sales_amount': 'sum',
    'transaction_id': pd.Series.nunique
}).reset_index()
sales_by_weekday['avg_transaction_value'] = sales_by_weekday['sales_amount'] / sales_by_weekday['transaction_id']

# กำหนดลำดับวันในสัปดาห์
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sales_by_weekday['weekday'] = pd.Categorical(sales_by_weekday['weekday'], categories=weekday_order, ordered=True)
sales_by_weekday = sales_by_weekday.sort_values('weekday')

print("รูปแบบการซื้อตามวันในสัปดาห์:")
display(sales_by_weekday)

# แสดงกราฟแท่งเปรียบเทียบจำนวนธุรกรรมตามวันในสัปดาห์
plt.figure(figsize=(12, 6))
sns.barplot(x='weekday', y='transaction_id', data=sales_by_weekday)
plt.title('จำนวนธุรกรรมตามวันในสัปดาห์')
plt.tight_layout()
plt.show()

In [ ]:
# แสดงกราฟเปรียบเทียบมูลค่าเฉลี่ยต่อธุรกรรมตามวันในสัปดาห์
plt.figure(figsize=(12, 6))
sns.barplot(x='weekday', y='avg_transaction_value', data=sales_by_weekday)
plt.title('มูลค่าเฉลี่ยต่อธุรกรรมตามวันในสัปดาห์')
plt.ylabel('มูลค่าเฉลี่ยต่อธุรกรรม (บาท)')
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์ความสัมพันธ์ระหว่างวันในสัปดาห์และกลุ่มอายุ
weekday_age_sales = sales_with_info.groupby(['weekday', 'age_group']).agg({
    'sales_amount': 'sum'
}).reset_index()

# กำหนดลำดับวันในสัปดาห์
weekday_age_sales['weekday'] = pd.Categorical(weekday_age_sales['weekday'], categories=weekday_order, ordered=True)
weekday_age_sales = weekday_age_sales.sort_values(['weekday', 'age_group'])

# แสดงกราฟความสัมพันธ์ระหว่างวันในสัปดาห์และกลุ่มอายุ
plt.figure(figsize=(14, 8))
weekday_age_pivot = weekday_age_sales.pivot(index='weekday', columns='age_group', values='sales_amount')
weekday_age_pivot.plot(kind='bar', stacked=True)
plt.title('ยอดขายตามวันในสัปดาห์และกลุ่มอายุ')
plt.xlabel('วันในสัปดาห์')
plt.ylabel('ยอดขาย (บาท)')
plt.legend(title='กลุ่มอายุ')
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์ความถี่ในการซื้อของลูกค้า
customer_frequency = sales_df.groupby('customer_id').agg({
    'transaction_id': pd.Series.nunique,
    'sales_amount': 'sum'
}).reset_index()
customer_frequency.columns = ['customer_id', 'num_transactions', 'total_spend']
customer_frequency['avg_transaction_value'] = customer_frequency['total_spend'] / customer_frequency['num_transactions']

# วิเคราะห์การกระจายของความถี่ในการซื้อ
plt.figure(figsize=(10, 6))
plt.hist(customer_frequency['num_transactions'], bins=20)
plt.title('การกระจายของความถี่ในการซื้อของลูกค้า')
plt.xlabel('จำนวนธุรกรรม')
plt.ylabel('จำนวนลูกค้า')
plt.tight_layout()
plt.show()

In [ ]:
# จัดกลุ่มลูกค้าตามความถี่ในการซื้อ
bins = [0, 1, 3, 5, 10, 100]
labels = ['ซื้อครั้งเดียว', '2-3 ครั้ง', '4-5 ครั้ง', '6-10 ครั้ง', 'มากกว่า 10 ครั้ง']
customer_frequency['frequency_group'] = pd.cut(customer_frequency['num_transactions'], bins=bins, labels=labels)

# วิเคราะห์การซื้อตามกลุ่มความถี่
frequency_group_analysis = customer_frequency.groupby('frequency_group').agg({
    'customer_id': 'count',
    'total_spend': 'mean',
    'avg_transaction_value': 'mean'
}).reset_index()
frequency_group_analysis.columns = ['frequency_group', 'num_customers', 'avg_total_spend', 'avg_transaction_value']

print("การวิเคราะห์ตามกลุ่มความถี่ในการซื้อ:")
display(frequency_group_analysis)

# แสดงกราฟสัดส่วนลูกค้าตามกลุ่มความถี่ในการซื้อ
plt.figure(figsize=(10, 6))
plt.pie(frequency_group_analysis['num_customers'], labels=frequency_group_analysis['frequency_group'], autopct='%1.1f%%')
plt.title('สัดส่วนลูกค้าตามกลุ่มความถี่ในการซื้อ')
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
# แสดงกราฟเปรียบเทียบค่าใช้จ่ายเฉลี่ยตามกลุ่มความถี่ในการซื้อ
plt.figure(figsize=(12, 6))
sns.barplot(x='frequency_group', y='avg_total_spend', data=frequency_group_analysis)
plt.title('ค่าใช้จ่ายเฉลี่ยตามกลุ่มความถี่ในการซื้อ')
plt.xlabel('กลุ่มความถี่ในการซื้อ')
plt.ylabel('ค่าใช้จ่ายเฉลี่ย (บาท)')
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์ความถี่ในการซื้อตามกลุ่มอายุและเพศ
customer_frequency_demographic = customer_frequency.merge(customers_df[['customer_id', 'age_group', 'gender']], on='customer_id')
frequency_by_demographic = customer_frequency_demographic.groupby(['age_group', 'gender']).agg({
    'num_transactions': 'mean',
    'total_spend': 'mean',
    'customer_id': 'count'
}).reset_index()
frequency_by_demographic.columns = ['age_group', 'gender', 'avg_frequency', 'avg_total_spend', 'num_customers']
frequency_by_demographic = frequency_by_demographic.sort_values(['age_group', 'gender'])

print("ความถี่ในการซื้อเฉลี่ยตามกลุ่มอายุและเพศ:")
display(frequency_by_demographic)

# แสดงกราฟเปรียบเทียบความถี่ในการซื้อตามกลุ่มอายุและเพศ
plt.figure(figsize=(12, 6))
freq_pivot = frequency_by_demographic.pivot(index='age_group', columns='gender', values='avg_frequency')
freq_pivot = freq_pivot.reindex(age_order)
freq_pivot.plot(kind='bar', width=0.7)
plt.title('ความถี่ในการซื้อเฉลี่ยตามกลุ่มอายุและเพศ')
plt.xlabel('กลุ่มอายุ')
plt.ylabel('จำนวนธุรกรรมเฉลี่ย')
plt.legend(title='เพศ')
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์รูปแบบการชำระเงิน
payment_analysis = sales_df.groupby('payment_method').agg({
    'sales_amount': 'sum',
    'transaction_id': pd.Series.nunique
}).reset_index()
payment_analysis['avg_transaction_value'] = payment_analysis['sales_amount'] / payment_analysis['transaction_id']
payment_analysis = payment_analysis.sort_values('sales_amount', ascending=False)

print("การวิเคราะห์รูปแบบการชำระเงิน:")
display(payment_analysis)

# แสดงกราฟวงกลมของสัดส่วนการชำระเงินตามยอดขาย
plt.figure(figsize=(10, 8))
plt.pie(payment_analysis['sales_amount'], labels=payment_analysis['payment_method'], autopct='%1.1f%%')
plt.title('สัดส่วนการชำระเงินตามยอดขาย')
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
# แสดงกราฟเปรียบเทียบค่าเฉลี่ยต่อธุรกรรมตามวิธีการชำระเงิน
plt.figure(figsize=(10, 6))
sns.barplot(x='payment_method', y='avg_transaction_value', data=payment_analysis)
plt.title('ค่าเฉลี่ยต่อธุรกรรมตามวิธีการชำระเงิน')
plt.ylabel('ค่าเฉลี่ยต่อธุรกรรม (บาท)')
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์ความสัมพันธ์ระหว่างวิธีการชำระเงินและกลุ่มอายุ
payment_by_age = sales_with_info.groupby(['payment_method', 'age_group']).agg({
    'sales_amount': 'sum'
}).reset_index()

# แสดงกราฟความสัมพันธ์ระหว่างวิธีการชำระเงินและกลุ่มอายุ
plt.figure(figsize=(14, 8))
payment_age_pivot = payment_by_age.pivot(index='payment_method', columns='age_group', values='sales_amount')
payment_age_pivot = payment_age_pivot[age_order]  # จัดเรียงคอลัมน์ตามลำดับกลุ่มอายุ
payment_age_pivot.plot(kind='bar', stacked=True)
plt.title('ยอดขายตามวิธีการชำระเงินและกลุ่มอายุ')
plt.xlabel('วิธีการชำระเงิน')
plt.ylabel('ยอดขาย (บาท)')
plt.legend(title='กลุ่มอายุ')
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์การซื้อสินค้าตามกลุ่มอายุและหมวดหมู่สินค้า
age_category_sales = sales_with_info.groupby(['age_group', 'category']).agg({
    'sales_amount': 'sum'
}).reset_index()

# แสดงกราฟความสัมพันธ์ระหว่างกลุ่มอายุและหมวดหมู่สินค้า
plt.figure(figsize=(14, 8))
age_category_pivot = age_category_sales.pivot(index='age_group', columns='category', values='sales_amount')
age_category_pivot = age_category_pivot.reindex(age_order)
age_category_pivot.plot(kind='bar', stacked=True)
plt.title('ยอดขายตามกลุ่มอายุและหมวดหมู่สินค้า')
plt.xlabel('กลุ่มอายุ')
plt.ylabel('ยอดขาย (บาท)')
plt.legend(title='หมวดหมู่สินค้า')
plt.tight_layout()
plt.show()

In [ ]:
# คำนวณสัดส่วนการซื้อสินค้าตามกลุ่มอายุ
age_category_pct = pd.DataFrame()
for age in age_order:
    age_data = age_category_sales[age_category_sales['age_group'] == age]
    total_sales = age_data['sales_amount'].sum()
    age_data['percentage'] = (age_data['sales_amount'] / total_sales) * 100
    age_category_pct = pd.concat([age_category_pct, age_data])

# แสดงกราฟสัดส่วนการซื้อสินค้าตามกลุ่มอายุ
plt.figure(figsize=(15, 10))
for i, age in enumerate(age_order):
    plt.subplot(2, 3, i+1)
    age_data = age_category_pct[age_category_pct['age_group'] == age]
    plt.pie(age_data['percentage'], labels=age_data['category'], autopct='%1.1f%%')
    plt.title(f'สัดส่วนการซื้อสินค้าของกลุ่มอายุ {age}')
    plt.axis('equal')
plt.tight_layout()
plt.show()